# Vertex AI Bitcoin Model - Prediction

### Dataset

The dataset we are using is the Bitcoin dataset from kaggle

### Objective

This notebook demonstrates, using the Vertex AI Python client library, how to train and make predictions on an AutoML timeseries model based on a timeseries dataset.

The steps performed include the following:

- Create a Vertex AI model training job.
- Train an AutoML Timeseries forecasting model.
- Deploy the `Model` resource to a serving `Endpoint` resource.
- Make a prediction by sending data.
- Undeploy the `Model` resource.

### Costs 

This uses billable components of Google Cloud:

* Vertex AI :  ~14 $/hour of training, will require 1 hour for training the forecasting jobs
* Cloud Storage : minimal
* Bigquery : minimal ( Free Tier Mostly )



## Installation

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

USER_FLAG = ""
# Google Cloud Notebook requires dependencies to be installed with '--user'
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

Install the latest version of the Vertex AI client library.

Run the following command in your virtual environment to install the Vertex SDK for Python:

In [ ]:
!pip3 install {USER_FLAG} google-cloud-aiplatform

Install the Cloud Storage library:

In [ ]:
! pip3 install {USER_FLAG} google-cloud-storage

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

Otherwise, set your project ID here.

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

### Authenticate your Google Cloud account

**If you are using Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This notebook demonstrates how to use Model Builder SDK to create an AutoML model based on a tabular dataset. You will need to provide a Cloud Storage bucket where the dataset will be stored.

Set the name of your Cloud Storage bucket below. It must be unique across all of your 
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Vertex AI services are
available](https://cloud.google.com/vertex-ai/docs/general/locations). You may
not use a Multi-Regional Storage bucket for training with Vertex AI.

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}
REGION = "[your-region]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME

### Copy dataset into your Cloud Storage bucket

In [ ]:
IMPORT_FILE = 'coin_Bitcoin.csv'
PREDICT_FILE = 'coin_Bitcoin_predict.csv'
! gsutil cp {IMPORT_FILE} {BUCKET_NAME}/data/
! gsutil cp {PREDICT_FILE} {BUCKET_NAME}/data/

gcs_source = f'{BUCKET_NAME}/data/{IMPORT_FILE}'

### Import Vertex SDK for Python

Import the Vertex SDK into your Python environment and initialize it.

In [ ]:
import os
import sys

from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

### Create a Managed Tabular Dataset from a CSV

This section will create a dataset from a CSV file stored on your GCS bucket.

In [ ]:
ds = dataset = aiplatform.TimeSeriesDataset.create(
    display_name='bitcoin-timeseries-dataset',
    gcs_source = gcs_source,
)

ds.resource_name

### Launch a Training Job to Create a Model

Once we have defined your training script, we will create a model. The `run` function creates a training pipeline that trains and creates a `Model` object. After the training pipeline completes, the `run` function returns the `Model` object.

In [ ]:
job = aiplatform.AutoMLForecastingTrainingJob(
    display_name='train-bitcoin-forecasting' ,
    optimization_objective = 'minimize-mae'
)

# This will take around an hour to run
model = job.run(
    dataset=ds,
    target_column='Open',
    time_column = 'Date',
    time_series_identifier_column = 'SNo',
    unavailable_at_forecast_columns = ['High','Low','Open','Close','Volume','Marketcap'],
    available_at_forecast_columns = ['date'],
    forecast_horizon = 30,
    context_window = 30,
    data_granularity_unit = 'day',
    data_granularity_count = 1,
    export_evaluated_data_items = True,
    budget_milli_node_hours = 100, # means 1 node hour - Cost will be ~14 dollars 
    model_display_name='bitcoin-forecast-model'
)

### Create a prediction job for your model

You use your model to make predictions, This function does two things:

1. Creates an `Job` resource 
2. Runs the `Job` resource 

### NOTE: Wait until the model **FINISHES** deployment before proceeding to prediction.

In [ ]:
predictJob = aiplatform.BatchPredictionJob (
    batch_prediction_job_name = 'bitcoin-predection-job'
)

prediction = predictJob.create(
    job_display_name = 'bitcoin-redict-job-run',
    model_name = 'bitcoin-forecast-model',
    instances_format = 'csv',
    predictions_format ='bigquery',
    gcs_source = f'{BUCKET_NAME}/data/{PREDICT_FILE}'
)

### Predict on the endpoint



# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used.

Otherwise, you can delete the individual resources you created :

- Training Job
- Model
- Cloud Storage Bucket

**Note**: You must delete any `Model` resources deployed to the `Endpoint` resource before deleting the `Endpoint` resource.

In [ ]:
delete_training_job = True
delete_model = True
delete_endpoint = True

# Warning: Setting this to true will delete everything in your bucket
delete_bucket = False

# Delete the training job
job.delete()

# Delete the model
model.delete()

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil -m rm -r $BUCKET_NAME